In [16]:
#importing required libraries
import warnings
warnings.filterwarnings("ignore")
#image precessing
import cv2
#file handling
import os
#obvious
import numpy as np
import pandas as pd
#library to know the progress of for loops
from tqdm import tqdm
# from matplotlib import pyplot as plt
from keras.utils import np_utils
from sklearn.utils import shuffle

In [17]:
#the path where the required files are
x = []
y = []
IMG_SIZE = 64
base_dir = 'hackday'
for i in tqdm(os.listdir(base_dir)):
    path = os.path.join(base_dir,i)
    print(str(i)+"class")
    for j in os.listdir(path):
        
        img = cv2.imread(os.path.join(path,j),0)
#         print(img.shape)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        img = img/255
        if img is not None:
            x.append(img)
            y.append(i)

  0%|          | 0/5 [00:00<?, ?it/s]

0class


 20%|██        | 1/5 [00:00<00:01,  3.67it/s]

4class


 40%|████      | 2/5 [00:00<00:00,  3.72it/s]

...

 60%|██████    | 3/5 [00:00<00:00,  3.79it/s]

2class


 80%|████████  | 4/5 [00:01<00:00,  3.93it/s]

3class


100%|██████████| 5/5 [00:01<00:00,  3.98it/s]


In [18]:
x = np.array(x)
y = np.array(y)

In [19]:
x.shape,y.shape

((5855, 64, 64), (5855,))

In [20]:
y = np_utils.to_categorical(y)
print(y.shape)

(5855, 5)


In [21]:
x,y = shuffle(x,y,random_state = 0)

In [22]:
x = x.reshape(x.shape[0],IMG_SIZE,IMG_SIZE,1)
x.shape,y.shape

((5855, 64, 64, 1), (5855, 5))

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

In [24]:
#multi classification model
nClasses = 5

In [25]:
#constructing the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Flatten())
model.add(Dense(nClasses, activation='softmax'))

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
__________

In [27]:
model.fit(x,y,epochs =2,batch_size = 100)

Epoch 1/2
5855/5855 [==============================] - 26s 4ms/step - loss: 0.1841 - acc: 0.9532
Epoch 2/2
5855/5855 [==============================] - 26s 4ms/step - loss: 0.0038 - acc: 0.9993


In [28]:
loss2, acc2 = model.evaluate(x,y)

5855/5855 [==============================] - 6s 1ms/step


In [29]:
acc2

1.0

In [30]:
model.save("Gesture-model-saved.h5")